In [ ]:
import numpy as np
import pandas as pd

import xmltodict
import json
import requests

In [ ]:
#read data from excel file into dataframe 'phospho_df'
phospho_df = pd.read_excel('~/Desktop/phosphoELM_all_2015-04.xlsm')

# This is a title
## This is a subtitle

hello you can write in **bold**, *italic*

lists:
- asd
- asd
- asd


numbered lists:
1. asd
2. 234
3. 46

In [ ]:
#select columns necessary for .gmt format and filter into new dataframe 'df'
df = phospho_df[['acc', 'kinases']]

#drop all columns with an 'NaN' value for the kinases
df.dropna(axis = 0, inplace = True)

#drop duplicate rows in the dataframe
df.drop_duplicates(inplace = True)

#set index of protein values 'acc' as kinases
#creates new dataframe 'kin'
kin = df.set_index('kinases')

#Create dictionary 'PhosphoELM' with kinases as keys
PhosphoELM = dict([(key, []) for key in kin.index])

#Group kinases in dataframe 'kin'
#Aggregate data in 'kin' according to kinase groups
#kin = kin.groupby('kinases').agg(lambda x: tuple(x))

#Create a new column 'PhosphoELM' as description of data
#kin.insert(0, 'Description', 'PhosphoELM')

In [152]:
ENRICHR_URL = 'https://www.ebi.ac.uk/proteins/api/proteins/%s'

def uniprot_to_gene(protein_id):
    response = requests.get(ENRICHR_URL % protein_id)
    if not response.ok:
        name = np.NaN
    else:
        data = xmltodict.parse(response.text)
        entry = data['entry']
        
        # check if entry contains 'gene'
        if 'gene' in entry.keys():
            names = data['entry']['gene']['name']
            if type(names) == list:
                name = list(names[0].values())[1]
            else:
                name = list(names.values())[1]
        else:
            
    return name

    
for key, row in kin.acc.iteritems():
    
    protein_id = '%s' %row
    print(row)
    geneS = uniprot_to_gene(protein_id)
    PhosphoELM[key] = PhosphoELM[key] + [geneS]
    
    
    #look into plotly

O08605
O14543
O14746
O14746
O14920
O15111
O15117
O15162
O15162
O15350
O15350
O15350
O15360
O15392
O15392
O15530
O15530
O15530
O43318
O43516
O43524
O43524
O43524
O54918
O54918
O54918
O60674
O75553
O96017
O96017
O96017
O96017
P00176
P00340
P00511
P00514
P00515
P00515
P00515
P00516
P00517
P00519
P00519
P00519
P00520
P00520
P00523
P00523
P00523
P00523
P00528
P00533
P00533
P00533
P00533
P00533
P00533
P00533
P00533
P00698
P00736
P00736
P00860
P01099
P01103
P01103
P01106
P01106
P01106
P01106
P01233
P01350
P01589
P02253
P02253
P0C0S9


KeyError: 'gene'

In [ ]:
# fix the dataframe in order to have three columns:
# kinases, description, acc_merged (acc, but all elements are joined by a \t symbol)
# with a reset index

#reset index of the datframe to integers, restores column 'kinases'
kin.reset_index(inplace = True)

#create column 'acc_merged' in which all 'acc' elements are joined by a \t symbol
kin['acc_merged'] = ['\t'.join(x) for x in kin['acc']]

acc = pd.Series(kin['acc'])

acc[4]

#drop the now-unneccesary column 'acc'
kin.drop('acc', axis=1, inplace = True)

#Create dictionary 'PhosphoELM' with kinases as keys
PhosphoELM = dict([(key, []) for key in kin.index])

# loop through rows with iterrows()
for index, rowData in kin.iterrows():
    line = ['\t'.join(rowData)]
    PhosphoELM[index] = line

In [ ]:


for index, rowData in acc.iteritems():
    for x in rowData:
        protein_id = str(x)
        print(x)
        response = requests.get(ENRICHR_URL % protein_id)
        data = response.text
        if not response.ok:
            continue
            #raise Exception('Error getting protein data from database')
        


In [ ]:
#Transfer tsv info into a new txt file
with open('PhosphoELM.txt', 'w') as openfile:
    for index in PhosphoELM:
        openfile.write(str(PhosphoELM[index]) + '\n')

In [ ]:
ENRICHR_URL = 'https://www.ebi.ac.uk/proteins/api/proteins/%s'

for index, rowData in acc.iteritems():
    for x in rowData:
        protein_id = str(x)
        print(x)
        response = requests.get(ENRICHR_URL % protein_id)
        data = response.text
        if not response.ok:
            continue
            #raise Exception('Error getting protein data from database')
        data = xmltodict.parse(response.text)
        data = data['entry']
        root_elements = data['gene']
        name_elements = root_elements['name']
        name = list(name_elements.values())[1]
        
#Create dictionary 'PhosphoELM' with kinases as keys
#PhosphoELM = dict([(key, []) for key in kin.index])

#Fill dict with values corresponding to 'PhosphoELM' and relevant proteins
#for key in kin.index:
   # PhosphoELM[key] = ([['PhosphoELM'], [kin.acc[key]]])